#### Pre-procesamiento de datos

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
import warnings
from statsmodels.tsa.seasonal import STL
import os
from functools import reduce
from sklearn.impute import KNNImputer

warnings.filterwarnings("ignore")

In [59]:
output_dir = "Preprocess_output"    

In [60]:
# Cargar los datos
df = pd.read_csv("Indicadores_Muestra.csv")
df["Fecha"] = pd.to_datetime(df["Fecha"])

In [61]:
df.head()

,Fecha,AFI,PMI,VGE,RBT,IMPB,CCI,IASS,EPA,PIB,IPI
0,1986-06-01,NaN,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN
1,1986-07-01,NaN,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN
2,1986-08-01,NaN,NaN,NaN,NaN,NaN,-6.3,NaN,NaN,NaN,NaN
3,1986-09-01,NaN,NaN,NaN,NaN,NaN,-7.1,NaN,NaN,NaN,NaN
4,1986-10-01,NaN,NaN,NaN,NaN,NaN,-7.2,NaN,NaN,NaN,NaN


#### Ajustes estacionales

De manera generalizada, se recomienda la implementación de JDemetra para la desestacionalización de las series. Para este trabajo, no obstante, se va a optar por el método LST, que se puede implementar en Python.

In [62]:
def limpieza_df(series_df, col):

    series_df = series_df[["Fecha", col]]

    primer_valido_df = series_df[col].first_valid_index()

    series_df = series_df.loc[primer_valido_df:]

    series_df = series_df.dropna().reset_index(drop=True)

    series_df  

    return series_df

In [63]:
def graficar_ajuste(result, data_og, columna):

    # Crear carpeta de salida si no existe

    os.makedirs(output_dir, exist_ok=True)

    print(columna)

    ig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10,8), sharex = True)

    ax1.plot(data_og[columna], color='black')

    plt.xticks(rotation=0)

    ax1.set_title("Serie original")

    ax2.plot(result.trend, color='black')

    ax2.set_title("Tendencia")

    ax3.plot(result.seasonal, color='black')

    ax3.set_title("Componente estacional")

    ax4.plot(result.resid, color='black')

    ax4.set_title("Componente residual")

    plt.tight_layout()

    plt.savefig(os.path.join(output_dir, f"{columna}_serie.png"))
    
    plt.close()

In [64]:
def ajustar_estacionalidad_stl(df, columna, periodo):
    """
    Aplica STL para eliminar componente estacional de una serie.
    
    Args:
        df (pd.DataFrame): con columnas 'Fecha' y la serie.
        columna (str): nombre de la columna de datos.
        periodo (int): 12 para mensual, 4 para trimestral.

    Returns:
        pd.DataFrame con columna ajustada
    """
    df = df.copy()
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    df.set_index("Fecha", inplace=True)

    serie = df[columna].dropna()

    stl = STL(serie, period=periodo)
    resultado = stl.fit()
    
    # Serie ajustada = tendencia 
    ajustada = resultado.trend + resultado.resid

    # Llamada a fucion

    graficar_ajuste(resultado, df, columna)

    # Unir en un nuevo DataFrame
    df_ajustada = pd.DataFrame({
        "Fecha": ajustada.index,
        f"{columna}_ajustada": ajustada.values
    })

    return df_ajustada


In [65]:
serie_impb = limpieza_df(df, "IMPB")

serie_pmi = limpieza_df(df, "PMI")

serie_cci = limpieza_df(df, "CCI")

In [66]:
impb_ajustada = ajustar_estacionalidad_stl(serie_impb, "IMPB", periodo=12)

IMPB


In [67]:
pmi_ajustada = ajustar_estacionalidad_stl(serie_pmi, "PMI", periodo=12)

PMI


In [68]:
cci_ajustada = ajustar_estacionalidad_stl(serie_cci, "CCI", periodo=12)

CCI


In [69]:
# Integración de IMPB

df = df.drop('IMPB', axis=1)

df.head()

,Fecha,AFI,PMI,VGE,RBT,CCI,IASS,EPA,PIB,IPI
0,1986-06-01,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN
1,1986-07-01,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN
2,1986-08-01,NaN,NaN,NaN,NaN,-6.3,NaN,NaN,NaN,NaN
3,1986-09-01,NaN,NaN,NaN,NaN,-7.1,NaN,NaN,NaN,NaN
4,1986-10-01,NaN,NaN,NaN,NaN,-7.2,NaN,NaN,NaN,NaN


In [70]:
impb_ajustada = impb_ajustada.rename(columns={"IMPB_ajustada": "IMPB"})

df_total = df.merge(impb_ajustada, on='Fecha', how='outer')

df_total.head()

,Fecha,AFI,PMI,VGE,RBT,CCI,IASS,EPA,PIB,IPI,IMPB
0,1986-06-01,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN,NaN
1,1986-07-01,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN,NaN
2,1986-08-01,NaN,NaN,NaN,NaN,-6.3,NaN,NaN,NaN,NaN,NaN
3,1986-09-01,NaN,NaN,NaN,NaN,-7.1,NaN,NaN,NaN,NaN,NaN
4,1986-10-01,NaN,NaN,NaN,NaN,-7.2,NaN,NaN,NaN,NaN,NaN


In [71]:
df_total.tail()

,Fecha,AFI,PMI,VGE,RBT,CCI,IASS,EPA,PIB,IPI,IMPB
463,2025-01-01,21399.164,57.3,133.0090,117.4045,NaN,129.417,22016.6,121.9704,100.622,2.534487e+07
464,2025-02-01,21457.899,54.9,134.5153,118.2299,NaN,129.461,NaN,NaN,101.371,2.539750e+07
465,2025-03-01,21480.979,56.2,134.1337,118.0460,NaN,129.534,NaN,NaN,102.293,2.594880e+07
466,2025-04-01,21550.139,54.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,2025-05-01,NaN,53.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Alineamiento temporal y tratamiento de variables no observadas

In [72]:
# Expandir series trimestrales con técnica "step-hold"

trimestrales = ["PIB", "EPA"]
for var in trimestrales:
    df_total[var] = df_total[var].fillna(method='ffill', limit=2)

In [73]:
df_total.tail()

,Fecha,AFI,PMI,VGE,RBT,CCI,IASS,EPA,PIB,IPI,IMPB
463,2025-01-01,21399.164,57.3,133.0090,117.4045,NaN,129.417,22016.6,121.9704,100.622,2.534487e+07
464,2025-02-01,21457.899,54.9,134.5153,118.2299,NaN,129.461,22016.6,121.9704,101.371,2.539750e+07
465,2025-03-01,21480.979,56.2,134.1337,118.0460,NaN,129.534,22016.6,121.9704,102.293,2.594880e+07
466,2025-04-01,21550.139,54.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,2025-05-01,NaN,53.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# Imputación de missings, entendidos como desalineamientos entre series

# Asegurar formato de fecha e índice temporal
df_total = df_total.copy()
df_total["Fecha"] = pd.to_datetime(df_total["Fecha"])
df_total.set_index("Fecha", inplace=True)

# Separar últimas 12 filas (con missing que quieres imputar)
df_rest = df_total.iloc[:-12]  # todo menos las últimas 12
df_tail = df_total.iloc[-12:]  # solo las últimas 12

# Aplicar imputación KNN solo a df_tail
imputer = KNNImputer(n_neighbors=5)

# Entrenar imputador con df_rest (sin missing) y usarlo para imputar df_tail
# Concatenamos ambos para que el imputador tenga contexto
df_imputacion = pd.concat([df_rest, df_tail])
df_imputed_array = imputer.fit_transform(df_imputacion)

# Convertimos el resultado a DataFrame
df_imputed = pd.DataFrame(df_imputed_array, columns=df_total.columns, index=df_imputacion.index)

# Recuperamos solo la parte imputada
df_tail_imputed = df_imputed.loc[df_tail.index]

# Volver a unir con el resto
df_final = pd.concat([df_rest, df_tail_imputed])

In [75]:
df_final.head()

,AFI,PMI,VGE,RBT,CCI,IASS,EPA,PIB,IPI,IMPB
Fecha,,,,,,,,,,
1986-06-01,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN,NaN
1986-07-01,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN,NaN
1986-08-01,NaN,NaN,NaN,NaN,-6.3,NaN,NaN,NaN,NaN,NaN
1986-09-01,NaN,NaN,NaN,NaN,-7.1,NaN,NaN,NaN,NaN,NaN
1986-10-01,NaN,NaN,NaN,NaN,-7.2,NaN,NaN,NaN,NaN,NaN


In [76]:
df_final.tail()

,AFI,PMI,VGE,RBT,CCI,IASS,EPA,PIB,IPI,IMPB
Fecha,,,,,,,,,,
2025-01-01,21399.1640,57.3,133.00900,117.40450,-9.46,129.4170,22016.60,121.97040,100.6220,2.534487e+07
2025-02-01,21457.8990,54.9,134.51530,118.22990,-9.46,129.4610,22016.60,121.97040,101.3710,2.539750e+07
2025-03-01,21480.9790,56.2,134.13370,118.04600,-9.24,129.5340,22016.60,121.97040,102.2930,2.594880e+07
2025-04-01,21550.1390,54.7,132.83694,117.38898,-14.02,128.3834,21952.72,121.69492,101.3568,2.540739e+07
2025-05-01,19310.8224,53.4,121.13408,105.25878,-15.62,103.3140,19876.36,111.37536,100.7980,2.518388e+07


In [78]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller

def test_adf_modular_trimestral(df, variables_trimestrales=[]):
    """
    Aplica transformaciones a series temporales.
    
    - Variables trimestrales (en variables_trimestrales):
        - Aplica log.
        - Calcula diferencia logarítmica entre trimestres (meses 1, 4, 7, 10).
        - Reparte la variación entre los tres meses del trimestre.
        - Se limita a datos hasta la última observación válida (no imputada).
    
    - Variables no trimestrales:
        - Aplica log si todos los valores son positivos.
        - Si el test ADF indica no estacionariedad, aplica una diferencia.
    
    Devuelve:
    - df_transformed: DataFrame con todas las series transformadas.
    - df_resultados_adf: resumen con transformaciones y p-valores ADF.
    """
    df_transformed = pd.DataFrame(index=df.index)
    resultados_adf = []

    # --- Procesar variables trimestrales ---
    for col in variables_trimestrales:
        
        # Solo hasta la última observación real (evitar valores imputados recientes)
        fecha_max_valida = df[col].last_valid_index()
        serie = df[col].loc[:fecha_max_valida].dropna()
        log_aplicado = False
        diferencia_aplicada = False
        p_valor_despues = None

        if (serie > 0).all():
            serie_log = np.log(serie)
            log_aplicado = True
            fechas = serie_log.index
            fechas_trimestrales = [f for f in fechas if f.month in [1, 4, 7, 10]]
            serie_suavizada = pd.Series(index=serie_log.index, dtype='float64')

            for i in range(1, len(fechas_trimestrales)):
                fecha_actual = fechas_trimestrales[i]
                fecha_anterior = fechas_trimestrales[i - 1]
                delta_log = serie_log[fecha_actual] - serie_log[fecha_anterior]
                delta_mensual = delta_log / 3

                for j in range(3):
                    fecha_target = fecha_anterior + pd.DateOffset(months=j)
                    if fecha_target in serie_log.index:
                        serie_suavizada.loc[fecha_target] = delta_mensual

            serie = serie_suavizada.sort_index()
            diferencia_aplicada = True

            if serie.dropna().nunique() > 1:
                p_valor_despues = adfuller(serie.dropna())[1]

            p_valor_inicial = adfuller(np.log(df[col].dropna()))[1]

            resultados_adf.append({
                "Variable": col,
                "Log aplicado": log_aplicado,
                "p-valor ADF (original)": round(p_valor_inicial, 4),
                "Diferencia aplicada": diferencia_aplicada,
                "p-valor ADF (diferenciada)": round(p_valor_despues, 4) if p_valor_despues else ""
            })

            df_transformed[col] = serie

    # --- Procesar el resto de variables ---
    for col in df.columns:
        if col in variables_trimestrales:
            continue

        serie = df[col].dropna()
        log_aplicado = False
        diferencia_aplicada = False
        p_valor_despues = None

        if (serie > 0).all():
            serie = np.log(serie)
            log_aplicado = True

        p_valor_inicial = adfuller(serie.dropna())[1]

        if p_valor_inicial > 0.05:
            serie = serie.diff().dropna()
            diferencia_aplicada = True

            if serie.dropna().nunique() > 1:
                p_valor_despues = adfuller(serie.dropna())[1]

        resultados_adf.append({
            "Variable": col,
            "Log aplicado": log_aplicado,
            "p-valor ADF (original)": round(p_valor_inicial, 4),
            "Diferencia aplicada": diferencia_aplicada,
            "p-valor ADF (diferenciada)": round(p_valor_despues, 4) if p_valor_despues else ""
        })

        df_transformed[col] = serie

    df_resultados_adf = pd.DataFrame(resultados_adf)
    return df_transformed, df_resultados_adf


In [80]:
df_transformed, df_resultados_adf = test_adf_modular_trimestral(df_final, ["EPA", "PIB"])
df_resultados_adf

,Variable,Log aplicado,p-valor ADF (original),Diferencia aplicada,p-valor ADF (diferenciada)
0,EPA,True,0.4665,True,0.025
1,PIB,True,0.2153,True,0.001
2,AFI,True,0.4365,True,0.037
3,PMI,True,0.0000,False,
4,VGE,True,0.1639,True,0.0
5,RBT,True,0.1609,True,0.4522
6,CCI,False,0.2141,True,0.0
7,IASS,True,0.4077,True,0.0
8,IPI,True,0.2830,True,0.0
9,IMPB,True,0.0009,False,


In [85]:
import pandas as pd

def generar_csvs_modelos(df, ruta_base=""):
    """
    Genera tres archivos CSV desde un DataFrame transformado, uno para cada modelo:
    
    - ARIMA: solo la variable 'PIB'
    - VAR: todas las variables excepto 'CCI'
    - SNARIMAX: todas las variables disponibles
    
    Args:
        df (pd.DataFrame): DataFrame transformado con columnas de variables, incluyendo 'PIB' y 'CCI'.
        ruta_base (str): ruta opcional para guardar los archivos CSV (por defecto en el directorio actual)
    
    Returns:
        tuple: DataFrames resultantes para ARIMA, VAR y SNARIMAX.
    """
    df = df.copy()
    
    # Asegurar que la columna 'Fecha' esté como índice si existe
    if "Fecha" in df.columns:
        df["Fecha"] = pd.to_datetime(df["Fecha"])
        df.set_index("Fecha", inplace=True)

    columnas = df.columns

    # --- ARIMA: solo 'PIB'
    df_arima = df[["PIB"]]
    df_arima.to_csv(f"{ruta_base}df_arima.csv")

    # --- VAR: todas excepto 'CCI'
    columnas_var = [col for col in columnas if col != "CCI"]
    df_var = df[columnas_var]
    df_var.to_csv(f"{ruta_base}df_var.csv")

    # --- SNARIMAX: todas las variables
    df.to_csv(f"{ruta_base}df_snarimax.csv")

    return df_arima, df_var, df  # Devuelve también los DataFrames en memoria

In [86]:
df_arima, df_var, df_snarimax = generar_csvs_modelos(df_transformed)

In [87]:
# Guardar los DataFrames en CSV para otros notebooks
df_transformed.to_csv("df_transformed.csv")